In [1]:
from gurobipy import GRB, Model, quicksum
import pandas as pd

In [3]:
# Leer datos

In [2]:
# Se crea el modelo
model = Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-09


In [ ]:
# Se crea la variable de decisión x_j siendo j un cereal y luego se actualiza el modelo para agregarla
x = model.addVars(cereales, vtype=GRB.CONTINUOUS, name="x_j")
model.update()

In [ ]:
# Definimos los cereales y nutrientes dependiendo del largo de los dataframes respectivos
cereales = range(df_nutricionales.shape[1])
nutrientes = range(df_nutricionales.shape[0])

In [ ]:
# Se agregan las respectivas restricciones al modelo:
## r1: Mezcla compuesta únicamente por cereales
model.addConstr((quicksum(x[j] for j in cereales)==1), name = "r1")
## r2: Se debe cumplir la proporción mínima de nutrientes
model.addConstrs((quicksum(x[j]*df_nutricionales.iloc[i][j] for j in cereales) >= limites_inferiores[i] for i in nutrientes), name = "r2")
## r3: Se debe cumplir la proporción máxima de nutrientes
model.addConstrs((quicksum(x[j]*df_nutricionales.iloc[i][j] for j in cereales) <= limites_superiores[i] for i in nutrientes), name = "r3")
## r4: Naturaleza de las variables
model.addConstrs((x[j] >= 0 for j in cereales), name = "r4")


In [ ]:
# Se define la función objetivo
model.setObjective(quicksum(x[j]*costos[j] for j in cereales), GRB.MINIMIZE)

# Se optimiza el modelo
model.optimize()

In [ ]:
# Se obtiene e imprimime el valor objetivo
obj_val = model.objVal